In [7]:
#네이버뉴스 글로벌 경제(3시 55분 시작)
from bs4 import BeautifulSoup
import requests
import os
import time
from datetime import datetime, timedelta

base_url = "https://news.naver.com/breakingnews/section/101/262?date="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 기사 저장 디렉토리
output_dir = r"C:\Users\cobe2\Documents\SeSAC\PJ\nnews_Global"
os.makedirs(output_dir, exist_ok=True)

# 기사 본문 크롤링 함수(1번)
def extract_and_save_articles(soup, date, iteration):
    li_tg = soup.select('ul.sa_list > li > div.sa_item_inner > div.sa_item_flex > div.sa_thumb > div.sa_thumb_inner > a')
    for i in li_tg:
        crawling_url = i.attrs['href']
        response = requests.get(crawling_url, headers=headers)
        crawling_soup = BeautifulSoup(response.text, 'html.parser')  # 해당 뉴스기사 링크의 html 정보 추출

        # 뉴스기사 개별 사이트로 들어가서 제목, 날짜(수정 날짜 포함), 언론사 있는 경우만 추출
        title_tag = crawling_soup.select_one('h2.media_end_head_headline')
        title = title_tag.text if title_tag else "No title Found"       
        date_li_tag = crawling_soup.select_one('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')
        formatted_date = date_li_tag.text.replace("오후", "").replace('.', '_').replace(':', '_').replace(' ', '').replace('오전', '') if date_li_tag else "No Date Found"
        date_mo_tag = crawling_soup.select_one('span.media_end_head_info_datestamp_time._ARTICLE_MODIFY_DATE_TIME')
        modify_date = date_mo_tag.text.replace("오후","").replace('.', '_').replace(':', '_').replace(' ','').replace('오전', '') if date_mo_tag else " "
        source_img_tag = crawling_soup.select_one('div.media_end_head_top a img')
        source = source_img_tag['alt'] if source_img_tag else "No Source Found"

        # 기사본문, 기자이름(이메일포함)이 있는 경우만 추출
        Nnews_tag = crawling_soup.select_one('#dic_area')
        Nnews = Nnews_tag.text.replace('\n', '').replace('\r', '').replace('\t', '') if Nnews_tag else "No Content Found"
        Writer_tag = crawling_soup.select_one('span.byline_s')
        Writer = Writer_tag.text if Writer_tag else "No Writer Found"
        # 날짜가 들어가 있는 파일이름을 text 파일로 저장
        file_path = os.path.join(output_dir, f"{formatted_date}_네이버뉴스(글로벌).txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(f'제목\n{title}\n입력\n{formatted_date}\n수정\n{modify_date}\nSource\n{source}\n내용\n{Nnews}\n링크\n{crawling_url}\n\n기자\n{Writer}')
        #print(f'Saved file: {file_path}')

# 특정 날짜에 있는 기사를 크롤링(2번)
def get_articles_for_date(date):
    current_url = f"{base_url}{date.strftime('%Y%m%d')}"
    response = requests.get(current_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    iteration = 0
    extract_and_save_articles(soup, date, iteration)

    while True:
        # "data-cursor"=기사 더보기 찾아서 다음 기사를 요청
        next_meta_div = soup.select_one('div.section_latest_article_CONTENT_LIST_PERSIST_META')
        
        if next_meta_div:
            next_cursor = next_meta_div.get('data-cursor', None)
        else:
            print(f"No more articles to load for {date.strftime('%Y-%m-%d')}.")
            break  # 커서가 없거나 커서를 눌렀는데 기사가 없을 경우 다음 날짜로 넘어감

        if next_cursor:
            # 다음 기사 요청을 위한 url 만들기
            next_url = f"{current_url}&data-cursor={next_cursor}"

            # request 보내기
            response = requests.get(next_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')

            # 기사 더보기로 생긴 새로운 기사들을 크롤링 후 저장하는 함수(1번)로 돌아간다 
            iteration += 1
            extract_and_save_articles(soup, date, iteration)

            time.sleep(1)
        else:
            print(f"No more data-cursor found for {date.strftime('%Y-%m-%d')}.")
            break  # data-cursor가 없으면 break

# 날짜 정하기
start_date = datetime.strptime("20230830", "%Y%m%d")
end_date = datetime.strptime("20230830", "%Y%m%d")

# 날짜별로 2번 함수를 작동시킴 
current_date = start_date
while current_date <= end_date:
    
    #print(f"Crawling articles for {current_date.strftime('%Y-%m-%d')}")
    get_articles_for_date(current_date)
    current_date += timedelta(days=1)

print("Crawling End")

No more articles to load for 2023-08-30.
Crawling End
